# feature set 1 chromosomes 2, 3, 4 + unknown

## In this notebook:

* single input dataframe which includes unknown labels 
* using feature set 2
* model: SVM (rbf) 
* only training/test split, no epochs 
* aucuracy assessement of SVM:  0.39 wow thats bad

In [1]:
#!pip install torch
#!pip install pandas
#!pip install numpy
#!pip install sklearn
#!pip install matplotlib

In [2]:
import torch
import pandas as pd
import numpy as np

# using sklearn goodies
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_roc_curve
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split


## pull in pickled dataframe:
df = pd.read_pickle("./pickled/chr2-4_unk_featureset1.pkl")
df.shape # 40905, 9


(40905, 9)

In [3]:
## Split into train/test
data_copy = df.copy()
df_train = data_copy.sample(frac=0.75, random_state=1)
df_test = data_copy.drop(df_train.index)

print ('Training data subset\n')
print (df_train.head(3))  # head default n=5, first 3 is enough
print ('\nTest data subset\n')
print (df_test.head(3))


## Get label:
train_labels = df_train.pop('Problematic')  # replace with clinvar when column is available
test_labels = df_test.pop('Problematic')

Training data subset

                       aapos TSL codonpos HUVEC_fitCons_score  \
chr pos       ref alt                                           
2   25244258  C   T      583   1        2            0.714379   
    178586706 C   T    19758   5        1            0.564101   
    47403295  G   A       35   1        2             0.56214   

                           ExAC_cnv.score          GDI LoFtool_score  \
chr pos       ref alt                                                  
2   25244258  C   T     -1.30301309757823     41.83604      1.50E-01   
    178586706 C   T                     0  74772.86558      9.71E-01   
    47403295  G   A    -0.391627327762734    376.88542      9.71E-03   

                       SORVA_LOF_MAF0.005_HetOrHom  Problematic  
chr pos       ref alt                                            
2   25244258  C   T                       0.001198            1  
    178586706 C   T                       0.017173            0  
    47403295  G   A        

In [4]:
classifier1 = SVC(kernel='rbf', gamma='scale',random_state=42, class_weight="balanced") # default for kernel and gamma. AUC = 0.94
#classifier2 = SVC(kernel='linear', gamma=10 ,random_state=42) # 0.94
#classifier3 = SVC(kernel='sigmoid', gamma=10 ,random_state=42) # 0.82
print("done making classifier")

done making classifier


In [5]:
classifier1.fit(df_train, train_labels)
print("done fitting")

doen fitting


In [7]:

### Run some stats to see if the model is doing well. 
y_true = test_labels.to_numpy()
y_pred = classifier1.predict(df_test)


print("y true", y_true) ## These are both numpy arrays now, why are the metrics returning arrays rather than single f
print("y predicted", y_pred)

# Confusion Matrix
from sklearn.metrics import confusion_matrix
print("Confusion matrix: \n", confusion_matrix(y_true, y_pred))

# Accuracy
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_true, y_pred))

# Recall
from sklearn.metrics import recall_score
recall = recall_score(y_true, y_pred, average=None)
print("Recall TP/(TP+FN):", recall)

# Precision
from sklearn.metrics import precision_score
precision = precision_score(y_true, y_pred, average=None)
print("Precision TP/(TP+FP):",precision)

# F1 score
F1 = 2 * (precision * recall) / (precision + recall)
print("F1:", F1)

y true [2 1 1 ... 0 0 0]
y predicted [1 1 1 ... 2 2 1]
Confusion matrix: 
 [[1758 4035  957]
 [ 191 2022  207]
 [ 237  506  196]]
Accuracy: 0.393312889504402
Recall TP/(TP+FN): [0.26044444 0.83553719 0.20873269]
Precision TP/(TP+FP): [0.8042086  0.30809081 0.14411765]
F1: [0.39346464 0.45018368 0.17050892]


## 